# Pacotes

In [1]:
import pandas as pd # Manuesar dataframe
import numpy as np
import re

import nltk
from nltk.corpus import stopwords # stopwords

# Machine Learning

from keras.preprocessing.sequence import pad_sequences
from keras import preprocessing
from keras.preprocessing.text import Tokenizer
from keras.layers import Embedding, Flatten, Dense, Activation
from keras.models import Sequential
from keras.utils import to_categorical
from keras.optimizers import rmsprop

from sklearn.model_selection import train_test_split

C:\Users\tarek\Anaconda3\envs\py35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Stopwords

In [2]:
nltk.download('stopwords')
stop = stopwords.words('portuguese')
stop.append('é')
stop.append('r')
stop.append('—')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tarek\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Banco de Dados

## G1

In [31]:
g1 = pd.read_csv('g1_final.csv', header = None) # importando
g1.columns = ['date','news'] # renomeando colunas
g1.date = pd.to_datetime(g1.date) # convertendo datas

g1.news = g1.news.apply(lambda x: ' '.join([word.lower() for word in x.split() if word.lower() not in (stop) and word.isalpha()])) # removendo stopwords
g1[:10]

pandas._libs.tslibs.timestamps.Timestamp

In [32]:
ibov = pd.read_csv('ibov.csv')
ibov = ibov.rename(columns={'Index':'date'})
ibov.date = pd.to_datetime(ibov.date)
ibov[:5]

,date,BVSP.Open,BVSP.High,BVSP.Low,BVSP.Close,BVSP.Volume,BVSP.Adjusted,d
0,2016-01-04,43349.0,43349.0,42125.0,42141.0,2976300.0,42141.0,NaN
1,2016-01-05,42139.0,42534.0,42137.0,42419.0,2557200.0,42419.0,0.006597
2,2016-01-06,42410.0,42410.0,41590.0,41773.0,3935900.0,41773.0,-0.015229
3,2016-01-07,41772.0,41772.0,40695.0,40695.0,4032300.0,40695.0,-0.025806
4,2016-01-08,40695.0,41218.0,40463.0,40612.0,3221600.0,40612.0,-0.002040


## CDS 

In [4]:
cds = pd.read_csv('cdsg1.csv') # importando
cds = cds.drop('Unnamed: 0',1) # removendo coluna
cds.columns = ['date','cds','delt'] # renomeando colunas
cds.date = pd.to_datetime(cds.date) # convertendo datas
cds['lag'] = cds.delt.shift()
cds[:10]

,date,cds,delt,lag
0,2017-02-09,234.150,NaN,NaN
1,2017-02-10,227.985,-0.026682,NaN
2,2017-02-13,217.133,-0.048770,-0.026682
3,2017-02-14,217.704,0.002626,-0.048770
4,2017-02-15,221.314,0.016446,0.002626
5,2017-02-16,228.015,0.029829,0.016446
6,2017-02-17,231.216,0.013941,0.029829
7,2017-02-20,231.390,0.000752,0.013941
8,2017-02-21,225.414,-0.026166,0.000752
9,2017-02-22,224.305,-0.004932,-0.026166


## Banco de Dados Pronto

In [34]:
df = pd.merge(g1, ibov, on='date') # juntando banco de dados

df.d.apply(lambda x:float(x)) # convertendo cds para float

# criando dummy para variação positiva

positivo = df.d.apply(lambda x: 1 if x > 0 else 0)
df['positivo'] = positivo
df.sample(10)

,date,news,BVSP.Open,BVSP.High,BVSP.Low,BVSP.Close,BVSP.Volume,BVSP.Adjusted,d,positivo
8163,2017-06-09,oficial menor resultado maio ficou índice acum...,62755.0,63066.0,62095.0,62211.0,3099900.0,62211.0,-0.008684,0
2464,2018-05-14,inscrições professores visitantes ufpb salário...,85222.0,86105.0,84688.0,85232.0,4080700.0,85232.0,0.000141,1
1261,2018-07-18,suspende liminar impedia leilão distribuidoras...,78124.0,78496.0,77164.0,77363.0,3394300.0,77363.0,-0.009817,0
8899,2017-05-09,passos abre seleção vagas área oportunidades p...,65533.0,66536.0,65533.0,66278.0,2679100.0,66278.0,0.011476,1
7799,2017-06-27,contas públicas continua diz ministro particip...,62188.0,62424.0,61580.0,61675.0,3275700.0,61675.0,-0.008249,0
1503,2018-07-02,duas semanas deputados senadores tentarão acor...,72763.0,72840.0,71935.0,72840.0,2753900.0,72840.0,0.001058,1
51,2018-10-01,superávit comercial bilhões brasil últimos diz...,79350.0,79557.0,78091.0,78624.0,3134400.0,78624.0,-0.009049,0
1524,2018-06-29,termina nesta prazo saque lote interrompido vo...,71779.0,73020.0,71779.0,72763.0,3879400.0,72763.0,0.013878,1
623,2018-08-29,aprovam convenções coletivas volta obrigatorie...,77472.0,78783.0,77399.0,78389.0,3127600.0,78389.0,0.011823,1
3077,2018-04-11,mil domicílios deixaram receber bolsa família ...,84494.0,85417.0,84340.0,85246.0,3370600.0,85246.0,0.008709,1


# Machine Learning

## Palavras para vetores

In [35]:
tokenizer = Tokenizer(num_words = 2000, lower = True)
tokenizer.fit_on_texts(df.news)

In [36]:
sequences = tokenizer.texts_to_sequences(df.news)
sequences = pad_sequences(sequences, maxlen = 400)

In [37]:
tokenizer.word_index

{'book': 10377,
 'lucon': 33408,
 'desemboca': 44323,
 'medica': 12796,
 'hamilton': 13424,
 'projetamos': 28224,
 'foster': 14646,
 'desvantagens': 15420,
 'julius': 21392,
 'modernizando': 36134,
 'suspenda': 11349,
 'continentes': 49992,
 'müssnich': 36523,
 'portalegre': 32785,
 'breach': 29695,
 'reclamaram': 8946,
 'fóssil': 17002,
 'porto': 635,
 'trimestreal': 33751,
 'secreto': 11838,
 'seráphico': 38823,
 'divindade': 26083,
 'federações': 11168,
 'econômicocade': 48473,
 'dirigiu': 16862,
 'tacitamente': 52607,
 'mercadoriafiscal': 36604,
 'sanseverino': 29543,
 'singelo': 29441,
 'dpu': 37150,
 'centralize': 40172,
 'uberlândia': 5467,
 'piranhas': 51468,
 'debochado': 35528,
 'gasômetro': 35314,
 'péssimas': 22084,
 'herbário': 47036,
 'coincidiram': 28716,
 'caterpillar': 15337,
 'neurologistamédico': 41888,
 'margrette': 22013,
 'liquidações': 17946,
 'aceitável': 11205,
 'vagasassistente': 14137,
 'demandou': 24874,
 'anosprojetos': 37403,
 'comemoração': 7454,
 'contra

## Dados

In [38]:
x, y = sequences, df.positivo
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

## Modelo

In [39]:
model = Sequential()
model.add(Embedding(2000, 100, input_length = 400)) 
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

rms = rmsprop(lr = 0.001)

model.compile(optimizer = rms, loss='binary_crossentropy', metrics=['acc'], )
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 400, 100)          200000    
_________________________________________________________________
flatten_1 (Flatten)          (None, 40000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               5120128   
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 33        
Total params: 5,330,497
Trainable params: 5,330,497
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(x_train, y_train, epochs=10, validation_data = (x_test, y_test), batch_size=150)
history

Train on 8525 samples, validate on 2132 samples
Epoch 1/10
8525/8525 [==============================] - 10s 1ms/step - loss: 0.7075 - acc: 0.5063 - val_loss: 0.6936 - val_acc: 0.4808
Epoch 2/10
8525/8525 [==============================] - 13s 1ms/step - loss: 0.6167 - acc: 0.6544 - val_loss: 0.7481 - val_acc: 0.5211
Epoch 3/10
8525/8525 [==============================] - 15s 2ms/step - loss: 0.2565 - acc: 0.8999 - val_loss: 1.0252 - val_acc: 0.5136
Epoch 4/10
8525/8525 [==============================] - 11s 1ms/step - loss: 0.0532 - acc: 0.9860 - val_loss: 1.4297 - val_acc: 0.5080
Epoch 5/10
8525/8525 [==============================] - 8s 923us/step - loss: 0.0228 - acc: 0.9940 - val_loss: 1.8080 - val_acc: 0.5267
Epoch 6/10
8525/8525 [==============================] - 9s 1ms/step - loss: 0.0133 - acc: 0.9966 - val_loss: 2.1738 - val_acc: 0.5286
Epoch 7/10
8525/8525 [==============================] - 9s 1ms/step - loss: 0.0080 - acc: 0.9970 - val_loss: 2.5872 - val_acc: 0.5216
Epoch 8/

In [19]:
teste = '''
os melhoram projeção para economia em 2019
Itaú espera alta do PIB de 2,5% no ano que vem e Selic inalterada em 6,5% ao ano; Bradesco passou a prever crescimento de 2,8%.
Por G1

09/11/2018 12h09  Atualizado há uma hora

  
Os dois maiores bancos privados do país melhoraram nesta sexta-feira (9) a previsão para o desempenho da economia em 2019. Embora estejam mais otimistas com o futuro da atividade, Itaú e Bradesco mantiveram as projeções para o Produto Interno Bruto (PIB) deste ano.

O Itaú projeta agora um crescimento 2,5% em 2019, acima dos 2% previstos anteriormente. Para 2018, a expansão esperada segue em 1,3%. Na avaliação do banco, a melhora é sustentada por condições financeiras mais expansivas.

Na revisão de cenário, o Itaú espera que a taxa básica de juros (Selic) deve permanecer inalterada em 6,5% não só no encontro de política monetária de dezembro como também em todo o ano de 2019. O banco prevê que a Selic subirá para 8% apenas em 2020, ao invés do segundo semestre do ano que vem, como previa antes. A taxa básica de juros está em 6,5% ao ano desde março de 2018.

"A mudança é consequência da perspectiva de condições financeiras mais expansionistas, em particular com juros de mercado mais baixos e preços de ativos (por exemplo, ações) mais elevados", informou o banco em relatório.

Bradesco prevê alta de 2,8% do PIB
A melhora das condições financeiras do país também fez com que o Bradesco subisse para 2,8%, de 2,5%, a previsão do PIB de 2019. Neste ano, a alta projetada continua em 1,1%.

"A economia brasileira encontra-se em uma posição cíclica favorável à retomada do crescimento. A inflação e os juros estão baixos; as famílias e empresas estão menos alavancadas; o déficit externo é reduzido e há grande ociosidade no mercado de trabalho e na indústria", informou o Bradesco.

O Bradesco espera que a Selic encerre este ano em 6,5% e que o aumento na taxa de juros comece a partir do segundo trimestre do ano que vem. Ao fim de 2019, os juros devem chegar a 8%.

A melhora das condições financeiras pode ser explicada pelo fim da incerteza eleitoral e pela expectativa de que o próximo governo consiga dar andamento ao ajuste fiscal. Com esse quadro, o real se valorizou nos últimos meses e houve uma melhora do patamar da bolsa de valores.

Inflação
No cenário do Itaú, o IPCA, o índice que baliza a meta perseguida pelo Banco Central, teve sua previsão para 2019 revisada para 4,2%, de 4,3% antes, bem perto do centro da meta de 4,25% ao ano, com margem de 1,5 ponto percentual para mais ou para menos. A estimativa para o fechamento de 2018 também foi revisada, de 4,5% para 4,2%, "basicamente por conta da queda recente no preço da gasolina".

Já o Bradesco manteve a projeção para o IPCA deste ano em m 4,4% para 2018 e 4,25% para 2019. O banco, no entanto, indica "algum viés de baixa" para este ano.

Câmbio
Já no caso do dólar, o Itaú reduziu a R$ 3,75 o dólar no final deste ano, de R$ 3,90 antes, "refletindo a percepção do mercado de menores incertezas rondando a implementação de reformas". Em 2019, no entanto, a instituição manteve o nível de R$ 3,90 anteriormente previsto.

O Bradesco manteve projeção de R$ 3,70 para os dois anos.
'''
tokens = tokenizer.texts_to_sequences([teste])
tokens = pad_sequences(tokens, maxlen = 400)

prediction = model.predict(np.array(tokens))

In [ ]:
prediction[0][0]